# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846864398270                   -0.70    4.8         
  2   -7.852324278977       -2.26       -1.53    1.0   25.6ms
  3   -7.852614677564       -3.54       -2.56    1.5   28.2ms
  4   -7.852646002003       -4.50       -2.88    2.5   35.4ms
  5   -7.852646496544       -6.31       -3.16    1.0   96.8ms
  6   -7.852646680324       -6.74       -4.11    1.2   27.9ms
  7   -7.852646686638       -8.20       -5.12    2.0   33.1ms
  8   -7.852646686726      -10.06       -5.56    1.8   30.8ms
  9   -7.852646686728      -11.66       -5.68    1.2   27.7ms
 10   -7.852646686730      -11.79       -6.79    1.0   26.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846879583667                   -0.70           4.8         
  2   -7.852551661120       -2.25       -1.64   0.80    2.0    280ms
  3   -7.852637796281       -4.06       -2.76   0.80    1.0   25.6ms
  4   -7.852646569638       -5.06       -3.31   0.80    2.0   31.7ms
  5   -7.852646678820       -6.96       -4.19   0.80    1.2   25.7ms
  6   -7.852646686527       -8.11       -4.83   0.80    1.8   30.0ms
  7   -7.852646686718       -9.72       -5.55   0.80    1.8    107ms
  8   -7.852646686730      -10.94       -6.68   0.80    2.0   30.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.387763e+01     4.100265e+00
 * time: 0.06656002998352051
     1     1.461007e+00     2.058485e+00
 * time: 0.30220603942871094
     2    -1.703875e+00     2.479936e+00
 * time: 0.3268890380859375
     3    -3.604539e+00     2.055749e+00
 * time: 0.3623771667480469
     4    -4.861020e+00     2.014965e+00
 * time: 0.3978590965270996
     5    -6.493761e+00     1.315029e+00
 * time: 0.4342069625854492
     6    -7.180420e+00     1.084283e+00
 * time: 0.470203161239624
     7    -7.414416e+00     1.625344e+00
 * time: 0.49544811248779297
     8    -7.503117e+00     1.384841e+00
 * time: 0.5206341743469238
     9    -7.588632e+00     2.611082e+00
 * time: 0.5460879802703857
    10    -7.668138e+00     2.439695e+00
 * time: 0.5716490745544434
    11    -7.731010e+00     7.454065e-01
 * time: 0.6191070079803467
    12    -7.745108e+00     1.330413e+00
 * time: 0.6554319858551025
    13    -7.808243e+00     5.525375e-01
 * time: 0.68084096

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846677794223                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645941494                   -1.64         
  2   -7.852646686730       -6.13       -3.72    2.16s
  3   -7.852646686730      -13.39       -7.27    184ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.3688426175611255e-7
|ρ_newton - ρ_scfv| = 1.8350179184636994e-7
|ρ_newton - ρ_dm|   = 1.265732483282986e-9
